# 19章 1つの名義変数で量的予測変数を予測する

## はじめに
- 本章で扱うのは、量的被予測変数と名義的予測変数のデータ構造
- 例
    - 政治的な党派関係　→ 収入を予測
    - 性行動のカテゴリ　→　寿命を予測（今回扱うこと）
- このような種類のデータ構造に関する伝統的な手法
    - 一要因分散分析（ANOVA）、または一元配置ANOVAと呼ばれている
    - これらの手法を用いて、予測変数によって、被予測変数の平均値に相違があるのかを調べる。
        - 政治的な党派ごとに収入が異なるか、など
- 今回紹介するベイジアンアプローチは、この伝統的な分散分析モデルを階層的に一般化したもの
- また、量的予測変数に伴う上位の名義的予測変数が存在する状況を考える。
    - このようなデータ構造については、伝統的な手法として共分散分析(ANCOVA)が用いられている。
    - こちらについても、伝統的なモデルの一般化を考える

- 1つの名義的予測変数の線形関数
- データのノイズを表すための、正規分布に従う高騰のリンク関数も含む

- 一要因分散分析、一元配置ANOVAについて説明できると良い。。
- 共分散分析も。。


## 19.1. 量的データの複数の群の表記

<img src="./img/fig19_1.png" width="500">

- 各群のデータは「中心傾向周りの」「ランダムな変動」として記述されるものを扱う
- 各群の中心傾向は「全体のベースライン($\beta_0$)からの偏向」として捉える
- なお、ここではデータは、標準偏差の等しい正規分布に従っているとする(後ほど、標準偏差が群ごとに異なる場合を考える)

- 各群ごとの予測値 $\mu$ は、下記の式で表される。全体のベースラインに群の変更を加えている
$$
\begin{aligned} \mu & = \beta _ { 0 } + \sum _ { j } \beta _ { [ j ] } x _ { [ j ] } \\ & = \beta _ { 0 } + \vec { \beta } \cdot \vec { x } \end{aligned}
$$

- また、全体のベースラインからのカテゴリの偏向は、足しあげると0になるようになっている。（ゼロ和条件）

$$
\sum _ { j } \beta _ { [ j ] } = 0
$$

- ゼロ和条件がないと完全ではない。ゼロ和条件を満たすようにするのは簡単(式19.3)

- 図19.1　で示した記述的モデルは、古典的なANOVAで用いた伝統的なもの。
- より一般的なモデルはベイズようソフトウェアで直接実装される。
    - 外れ値は正規分布の代わりに裾の重いノイズ分布(t分布など)を用いることによって取り込み可能
    - 異なる群に異なる標準偏差を与えることができる

## 19.2. 伝統的な分散分析
- 「分散分析」：データ全体の分散を郡内分散と群間分散に分解するという意味
- 代数的にいうと、
    - 「全体平均値からの偏差の二乗和」＝「それぞれの群平均値からの偏差の二乗和」＋「全体平均値からの群平均の偏差の二乗和」
    - つまり、「全体の分散」＝「郡内分散」＋「群間分散」
    - ANOVAはこれらの代数的な側面を表している。
- ベイズ的手法は、要素の分散を推定できはするが、このような代数的な関係は使わない。
- そのため、ベイジアンアプローチとANOVAは異なるものである。しかし、類似はしている。

### 伝統的なANOVAについて
- 帰無仮説：各群の平均は等しい（なお、ANOVAは3群以上の場合に用いる）
- p値に基づいて上記を決定する
- p値は、帰無仮説からの擬似サンプリングによって計算される
- なお、伝統的なANOVAでは、帰無仮説は下記を前提としている
    1. 各群のデータが正規分布に従っている
    2. 各群、同じ標準偏差であること（分散の等質性）
- サンプリングする統計量は、郡内分散と群間分散の比。これをF比と呼ぶ。サンプリング分布はF分布と呼ばれている。
- この2つの前提は慣例として定着しているため、本書の基本モデルでも前提として持ってくる。
- ベイズ用ソフトウェアではこれらの前提は緩和される。
    - 各群によって異なる分散を用いることができる
    - 正規分布以外も郡内のデータを表すのに用いることができる

# 19.3. 階層ベイジアン・アプローチ
- 再掲
<img src="./img/fig19_1.png">

- 私たちの目標：ベイズ的な枠組みで、上述したパラメータを推定することである
- 式19.1.
$$
\begin{aligned} \mu & = \beta _ { 0 } + \sum _ { j } \beta _ { [ j ] } x _ { [ j ] } \\ & = \beta _ { 0 } + \vec { \beta } \cdot \vec { x } \end{aligned}
$$

- そのため、全てのパラメータは図19.2のように、構造化された事前分布を与えられる必要がある。

### 階層的ダイアグラム
<img src="./img/fig19_2.png" width="500">

- 下から見ていく
- データ $y_i$が予測値 $\mu_i$を中心に、正規分布に従っている。その正規分布の標準偏差には$\sigma_y$には一様事前分布が与えられている
- ベースラインパラメータ $\beta_0$はデータにとって広い範囲の正規事前分布を与える
- 群の偏向パラメータ $\beta_j$には平均0の正規事前分布を与える
    - なぜなら偏向パラメータの総和は0になるため。

- ベイジアンアプローチでの目新しいところは $\sigma_{\beta}$ 。それを偏向パラメータ分布の標準偏差とすること。
- 事前分布を何か選ぶこともできるし、定数として設定することも可能。
    - $\sigma_{\beta}$を大きな定数として設定することで、伝統的なANOVAにもっとも近い結果となる
- ???

### 19.3.1. JAGSでの実装


### 19.3.2. 例：セックスと死亡について
- ショウジョウバエでの実験
- オスについて性行為がどう負担になるかを調べる。
- 5つの群について、25匹ずつのオスのショウジョウバエが割り当てられた
    - ゼロ：相手がいないオスの群
    - 妊娠1：妊娠したメス1匹のみとつがいにされたオスの群
    - 妊娠8：妊娠したメス8匹とつがいにされた群
    - 処女1：未交尾のメス1匹のみとつがいにされた群
    - 処女8：未交尾のメス8匹とつがいにされた群
- 図19.3を見ると、性行為が増えるに従い、寿命が減るということを示唆している(図19.3 の赤丸)
- 目標：寿命と群間の差異の程度を推定すること。

<img src="./img/fig19_3_1.png">

- 図19.2　の階層的ダイアグラムを元に、事後予測分布を求めたのが図19.3
- MCMCチェインのある点における「群の平均」と「標準偏差」を持つ正規曲線を繰り返しプロットした。
- この正規曲線を髭と呼んでいる
    - 髭がもじゃもじゃしている：正規曲線がばらけていて、線がたくさん見える状態
    - 髭がもじゃもじゃしていない：正規曲線がまとまっていて、線が少なく見える状態
- 髭のもじゃもじゃが推定の不確実性を表す。その振れ幅が標準偏差の値を表している。
- 横幅の長い、薄い髭：高い確実性で推定される大きな標準偏差
- 横幅の短い、もじゃもじゃした髭：低い確実性で推定される小さな標準偏差
    - つまり、髭のもじゃもじゃは、平均と標準偏差の不確実性によって影響を受ける

- 結果の評価
    - 事後予測分布にそこまで大きな外れ値はなく、各群内でのデータの散らばりは事後正規分布の幅に合致していることを明らかにしている
        - 分散の等質性を目視で確認する場合、データの散らばりがどう見えるかはサンプルサイズによるため注意が必要
    - 正規分布の頂点で示された群平均の範囲を見ると、
        - 処女8群は他の群より低い
        - 処女1群も統制群よりも低そう
        - これらを確かめるために群平均の差を分析する

### 19.3.3. 対比
- 研究者は、群間の比較か、群の集合と集合の比較に興味を持っている。
    - 4つの操作群とゼロ統制軍の違い
    - 3つの統制群（ゼロ、妊娠1、妊娠8）の平均値と、2つの性行為が活発な群（処女1、処女8）
- MCMCチェインの各ステップから群の平均値の組をえることができる。
- もし群１と群２の平均の差に興味があれば、MCMCチェインの各ステップで計算すれば良い
$$
\begin{aligned} \mu _ { 1 } - \mu _ { 2 } & = \left( \beta _ { 0 } + \beta _ { 1 } \right) - \left( \beta _ { 0 } + \beta _ { 2 } \right) \\ & = ( + 1 ) \cdot \beta _ { 1 } + ( - 1 ) \cdot \beta _ { 2 } \end{aligned}
$$
- ベースラインは計算から除外され、群間の偏向は重み付けされた群偏差の和となる。
- もし群１〜３の平均値と群４〜５の平均値の差に興味があれば下記。
$$
\begin{aligned} \left( \mu _ { 1 } + \right. & \mu _ { 2 } + \mu _ { 3 } ) / 3 - \left( \mu _ { 4 } + \mu _ { 5 } \right) / 2 \\ & = \left( \left( \beta _ { 0 } + \beta _ { 1 } \right) + \left( \beta _ { 0 } + \beta _ { 2 } \right) + \left( \beta _ { 0 } + \beta _ { 3 } \right) \right) / 3 - \left( \left( \beta _ { 0 } + \beta _ { 4 } \right) + \left( \beta _ { 0 } + \beta _ { 5 } \right) \right) / 2 \\ & = \left( \beta _ { 1 } + \beta _ { 2 } + \beta _ { 3 } \right) / 3 - \left( \beta _ { 4 } + \beta _ { 5 } \right) / 2 \\ & = ( + 1 / 3 ) \cdot \beta _ { 1 } + ( + 1 / 3 ) \cdot \beta _ { 2 } + ( + 1 / 3 ) \cdot \beta _ { 3 } + ( - 1 / 2 ) \cdot \beta _ { 4 } + ( - 1 / 2 ) \cdot \beta _ { 5 } \end{aligned}
$$
- 群間の差は、重み付けされた群偏向の和。
- 群偏向の係数は足すとゼロ（上だと、$1/3,1/3,1/3,-1/2,-1/2$）。このような組を対比と呼ぶ。（正の係数、負の係数をそれぞれ足すと$＋１，ー１$になる）
- ここで求めた差をチェインの各ステップで $\sigma_y$で割ると効果量として表現できる。(よくわからない)

<img src="./img/fig19_3_2.png">

- plotMCMCの出力のいくつかが図19.3の下部（対比の事後分布）
- ２段あるうちの上段は差の事後分布、下段は効果量の事後分布
- 差の事後分布は寿命（日数）のスケールで表現されている。
- ROPE：実践的に等価な範囲(p342 12章)
    - 特定の状況において、パラメータの値が空値(帰無仮説が主張する値)と実質的に等価であるとみなされる小さな範囲のこと
    - ここでは差は０日、ROPEの範囲は−１．５日〜１．５日としている。
    - ROPE全体が、パラメータの事後分布の95%HDIの外側にある場合、そのパラメータの帰無仮説は棄却される
    - ROPE全体に、９５％HDIが完全に内包されていた場合、パラメータの帰無仮説は実用的な目的に合ったものとして受け入れられる
- 統制群の平均と性に活発な群（一番右端の列）について見る。
    - 平均値の差は、最頻値で１５日と大きい
    - ROPEとは重なっていない。そのため帰無仮説０日は棄却される。
    - 効果量は1.0（これが大きいのかどうかよくわからない…）
- 統制群と処女１群(左から２列目)について見る。
    - 平均値の差の中央値は約７日。
    - 効果量は０．５をわずかに下回る。
    -  ROPEと重なっている。
     

#### 伝統的な分散分析
- いわゆる多重対比と呼ばれる検定が多かった
- 「すべての群が同時に正確に等しい」ということが本当かどうかを問うもの
- しかし、「すべての群が等しい」という帰無仮説が棄却された場合、次に気になるのはそれぞれの群の比較。
- 「すべての群が等しい」という帰無仮説が棄却されなくても、一部の群間には有意な差があるかもしれない。そうなると個別の群について検定する必要がある。
- ここまで用いた階層ベイズ推定はすべての群の、有意義な比較検定をするというもの。伝統的な分散分析の多重対比とは異なる。

### 19.3.4. 多重比較と縮小
- 